In [1]:
# กรณีรันครั้งแรกให้สร้าง user/password สำหรับติดต่อกับ sharepoint ตาม code ชุดนี้ ใช้ user ที่เข้า windows บริษัท
# import keyring

# keyring.set_password("sharepointonline_username", "username", "pongsak.b@irpc.co.th")
# keyring.set_password("sharepointonline_pass","password", "xxxx")

# print(keyring.get_password("sharepointonline_username", "username"),keyring.get_password("sharepointonline_pass","password"))

In [2]:
import keyring
import pandas as pd
import os
import io
import inspect
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
sharepoint_user = keyring.get_password("sharepointonline_username", "username")
sharepoint_pass = keyring.get_password("sharepointonline_pass","password")

baseurl = 'https://irpcplc.sharepoint.com'
basesite = '/sites/PREM/DSLV2/' # every share point has a home.
docurl = 'Shared%20Documents/P&ID_from_AutoCAD_&_Label/UHV_Label/'
siteurl = baseurl + basesite 
target_url = basesite+docurl
ctx_auth = AuthenticationContext(siteurl)
ctx_auth.acquire_token_for_user(sharepoint_user, sharepoint_pass)
ctx = ClientContext(siteurl, ctx_auth) # make sure you auth to the siteurl.


In [3]:
csv_folder = 'Shared%20Documents/P&ID_from_AutoCAD_&_Label/UHV_Label/'
pdf_folder = 'Shared%20Documents/P&ID_from_AutoCAD_&_Label/UHV_PDF/'

In [4]:
# https://stackoverflow.com/questions/67888440/retrieving-list-of-xlsx-files-from-sharepoint-folder-with-python
#ไฟล์ csv
def list_sharepoint_files(ctx,relativeUrl):    
    lists = ctx.web.get_folder_by_server_relative_url(relativeUrl)
    items = lists.files
    
    ctx.load(items)
    ctx.execute_query()
    
    files = []
    for item in items:
        files.append(item.properties['Name'])
    return files

In [5]:
# https://stackoverflow.com/questions/67888440/retrieving-list-of-xlsx-files-from-sharepoint-folder-with-python
#Folders
def list_sharepoint_folders(ctx,relativeUrl):    
    lists = ctx.web.get_folder_by_server_relative_url(relativeUrl)
    items = lists.folders
    
    ctx.load(items)
    ctx.execute_query()
    
    files = []
    for item in items:
        files.append(item.properties['Name'])
    return files

In [6]:
csv_files = list_sharepoint_files(ctx,csv_folder)
# csv_files

In [7]:
pdf_subfolder = list_sharepoint_folders(ctx,pdf_folder)
pdf_subfolder.sort()
# pdf_subfolder

In [8]:
# read all csv files to df
for filename in csv_files:
    # filename = 'UHV Plant-Engineering Items.csv'
    relative_url = basesite+csv_folder+filename
    response = File.open_binary(ctx, relative_url)
    
    #save data to BytesIO stream
    bytes_file_obj = io.BytesIO()
    bytes_file_obj.write(response.content)
    bytes_file_obj.seek(0) #set file object to start
    
    #read file into pandas dataframe
    df = pd.read_csv(bytes_file_obj, low_memory=False)
    print(len(df.columns))

18
11
14
22
12
18
18
22
7
17
8
11
15
12
12
17
11
10
7
12
7
11
18
22
11
14
16
19
6
17
17
7
9
12
7
17
17
7
11
6
17
7
18
11
7
18
22
22
7
11
16
7
18
18
18
22
17
16
11
17
18
11
7
7
7
16
6
11
17
18
15
7
14
15
11
7
18
17
16
33
12
12
19
22
7
14
16
14
7
18
19
6
18
7
17
14
11
6
4
14
11
19
7
7
17
19
11
14
17
17
7
7
12
17
18
19
18
7
19
17
14
17
16
7
14
7
7
7
17
17
9
14
15
12
22
11
12
7
7
15
15
17
17
14
6
7
15
12
7
17
15
12
6
17
14
18
11
7
12
7
7
16
7
7
7
15
12
7
15
7
18
17
14


In [9]:
# https://stackoverflow.com/questions/48424045/how-to-read-sharepoint-online-office365-excel-files-in-python-with-work-or-sch
# read data from sharepoint and save to local disk
# filename = 'UHV Plant-Engineering Items.csv'
# relative_url = basesite+docurl+filename
# with open(filename, 'wb') as output_file:
#     response = File.open_binary(ctx, relative_url)
#     output_file.write(response.content)

In [10]:
# https://stackoverflow.com/questions/48424045/how-to-read-sharepoint-online-office365-excel-files-in-python-with-work-or-sch
# read data from sharepoint and save data to BytesIO stream
# filename = 'UHV Plant-Engineering Items.csv'
# relative_url = basesite+csv_folder+filename
# response = File.open_binary(ctx, relative_url)

# #save data to BytesIO stream
# bytes_file_obj = io.BytesIO()
# bytes_file_obj.write(response.content)
# bytes_file_obj.seek(0) #set file object to start

# #read file into pandas dataframe
# df = pd.read_csv(bytes_file_obj, low_memory=False)

In [26]:
pdf_files = []
for pdfsubfolder in pdf_subfolder:
    print(pdfsubfolder)
    pdf_files.append(list_sharepoint_files(ctx,pdf_folder+pdfsubfolder))

Unit 00, 01
Unit 51
Unit 52
Unit 53
Unit 54
Unit 55
Unit 56
Unit 57
Unit 58
Unit 59
Unit 70
Unit 71
Unit 72
Unit 73
Unit 74
Unit 75
Unit 77
Unit 78
Unit 79
Unit 80


In [ ]:
pdf_files

[['9999-P-01-01-001.pdf',
  '9999-P-01-01-015.pdf',
  '9999-P-01-01-004.pdf',
  '9999-P-01-01-017.pdf',
  '9999-P-01-01-019.pdf',
  '9999-P-01-01-041.pdf',
  '9999-P-01-01-023.pdf',
  '9999-P-01-01-025.pdf',
  '9999-P-01-01-008.pdf',
  '9999-P-01-01-029.pdf',
  '9999-P-01-01-030.pdf',
  '9999-P-01-00-036.pdf',
  '9999-P-01-01-016.pdf',
  '9999-P-01-01-018.pdf',
  '9999-P-01-01-037.pdf',
  '9999-P-01-01-020.pdf',
  '9999-P-01-01-039.pdf',
  '9999-P-01-01-013.pdf',
  '9999-P-01-01-022.pdf',
  '9999-P-01-01-036.pdf',
  '9999-P-01-00-126.pdf',
  '9999-P-01-01-038.pdf',
  '9999-P-01-01-011.pdf',
  '9999-P-01-01-009.pdf',
  '9999-P-01-00-131.pdf',
  '9999-P-01-01-024.pdf',
  '9999-P-01-01-021.pdf',
  '9999-P-01-01-028.pdf',
  '9999-P-01-01-014.pdf',
  '9999-P-01-01-002.pdf',
  '9999-P-01-01-040.pdf',
  '9999-P-01-01-010.pdf',
  '9999-P-01-01-012.pdf',
  '9999-P-01-01-003.pdf',
  '9999-P-01-01-006.pdf',
  '9999-P-01-01-005.pdf',
  '9999-P-01-01-007.pdf',
  '9999-P-01-01-026.pdf',
  '9999-P-01

In [31]:
pdf_files_flatten = [item for sublist in pdf_files for item in sublist]

In [32]:
pdf_files_flatten

['9999-P-01-01-001.pdf',
 '9999-P-01-01-015.pdf',
 '9999-P-01-01-004.pdf',
 '9999-P-01-01-017.pdf',
 '9999-P-01-01-019.pdf',
 '9999-P-01-01-041.pdf',
 '9999-P-01-01-023.pdf',
 '9999-P-01-01-025.pdf',
 '9999-P-01-01-008.pdf',
 '9999-P-01-01-029.pdf',
 '9999-P-01-01-030.pdf',
 '9999-P-01-00-036.pdf',
 '9999-P-01-01-016.pdf',
 '9999-P-01-01-018.pdf',
 '9999-P-01-01-037.pdf',
 '9999-P-01-01-020.pdf',
 '9999-P-01-01-039.pdf',
 '9999-P-01-01-013.pdf',
 '9999-P-01-01-022.pdf',
 '9999-P-01-01-036.pdf',
 '9999-P-01-00-126.pdf',
 '9999-P-01-01-038.pdf',
 '9999-P-01-01-011.pdf',
 '9999-P-01-01-009.pdf',
 '9999-P-01-00-131.pdf',
 '9999-P-01-01-024.pdf',
 '9999-P-01-01-021.pdf',
 '9999-P-01-01-028.pdf',
 '9999-P-01-01-014.pdf',
 '9999-P-01-01-002.pdf',
 '9999-P-01-01-040.pdf',
 '9999-P-01-01-010.pdf',
 '9999-P-01-01-012.pdf',
 '9999-P-01-01-003.pdf',
 '9999-P-01-01-006.pdf',
 '9999-P-01-01-005.pdf',
 '9999-P-01-01-007.pdf',
 '9999-P-01-01-026.pdf',
 '9999-P-01-01-035.pdf',
 '9999-P-01-01-027.pdf',
